In [1]:
from fastai.vision.all import *

def predict_batch(self, item, rm_type_tfms=None, with_input=False):
    dl = self.dls.test_dl(item, rm_type_tfms=rm_type_tfms, num_workers=0)
    inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
    i = getattr(self.dls, 'n_inp', -1)
    inp = (inp,) if i==1 else tuplify(inp)
    dec = self.dls.decode_batch(inp + tuplify(dec_preds))
    dec_inp,dec_targ = map(detuplify, [dec[:i],dec[i:]])
    res = dec_targ,dec_preds,preds
    if with_input: res = (dec_inp,) + res
    return res
Learner.predict_batch = predict_batch
from fastai.callback.tensorboard import TensorBoardCallback

from sklearn import metrics, model_selection, preprocessing
from sklearn.model_selection import StratifiedKFold

In [2]:
def asl_name_fn(x):
    return Path(x).parent.name

In [3]:
config = {
    "experiment_name": "test_asl_starter",
    "ds_path": Path("/mnt/e/Datasets/asl/asl_alphabet_train/asl_alphabet_train"),
    "ds_name": "asl",
    "name_fn": asl_name_fn,
    "image_size": 224,
    "batch_size": 64,
    "epoch_steps": [1, 2],
    "enable_proxy_attention": True,
    # "change_subset_attention": tune.loguniform(0.1, 0.8),
    "change_subset_attention": .5,
    "validation_split": 0.3,
    # "shuffle_dataset": tune.choice([True, False]),
    "shuffle_dataset": True,
    "num_gpu": 1,
    "transfer_imagenet": False,
    "subset_images": 8000,
    # "proxy_threshold": tune.loguniform(0.008, 0.01),
    "proxy_threshold":  0.005,
    # "pixel_replacement_method": tune.choice(["mean", "max", "min", "black", "white"]),
    "pixel_replacement_method": "half",
    "model": "resnet18",
    # "proxy_steps": tune.choice([[1, "p", 1], [3, "p", 1], [1, 1], [3,1]]),
    # "proxy_steps": tune.choice([["p", 1],[1, 1], ["p",1], [1, "p",1], [1,1,1]]),
    "proxy_steps": ["p",1],
    "global_run_count" : 0,
    "gradient_method" : "saliency", #guidedgradcam
}


In [4]:
if config["subset_images"] != None:
    def get_image_files(path, recurse=True, folders=None):
        "Get image files in `path` recursively, only in `folders`, if specified."
        return get_files(path, extensions=image_extensions, recurse=recurse, folders=folders)[:config["subset_images"]]

In [5]:
dblock = DataBlock(blocks    = (ImageBlock, CategoryBlock),
                   get_items = get_image_files,
                   get_y     = asl_name_fn,
                   splitter  = RandomSplitter(),
                   item_tfms = Resize(config["image_size"]),
                   batch_tfms = [*aug_transforms(), Normalize.from_stats(*imagenet_stats)]
                   )

In [6]:
dls = dblock.dataloaders(config["ds_path"])

In [7]:
print(f"There are {dls.c} classes")

There are 3 classes


In [8]:
# dls.show_batch()

In [9]:
model_map = {
    "resnet18" : resnet18
}

In [10]:
cbs = [
    TensorBoardCallback(projector=True)
       ]

In [11]:
learn = vision_learner(dls, model_map[config["model"]], metrics=accuracy, pretrained=False, cbs = cbs)

/home/eragon/miniconda3/envs/pytorcher/envs/fai/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/eragon/miniconda3/envs/pytorcher/envs/fai/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [12]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.490132,0.059780,0.982500,01:21


In [34]:
test_im = get_image_files(config["ds_path"])

In [147]:
dls.train

In [148]:
preds = learn.get_preds(dl = dls.valid)

In [129]:
label_map = {i: l for i, l in enumerate(dls.vocab)}
rev_label_map = {l: i for i, l in enumerate(dls.vocab)}

config["label_map"]= label_map
config["rev_label_map"]= rev_label_map


In [150]:
original_labels = [config["rev_label_map"][asl_name_fn(x)] for x in dls.valid_ds.items]

In [152]:
preds[1]

tensor([1, 2, 2,  ..., 0, 2, 1])

In [156]:
wrong_indices = (torch.Tensor(original_labels) != preds[1]).nonzero()

In [157]:
wrong_indices

tensor([], size=(0, 1), dtype=torch.int64)

In [143]:
# preds = learn.predict_batch(train["image_id"][:10].values)

In [145]:
# preds[1]

TensorBase([20,  5, 13, 27,  4,  0,  1, 10, 18, 26])

In [13]:
from fastai.callback.captum import CaptumInterpretation

In [14]:
captum=CaptumInterpretation(learn)

In [25]:
dls.train_ds

(#6400) [(PILImage mode=RGB size=200x200, TensorCategory(1)),(PILImage mode=RGB size=200x200, TensorCategory(1)),(PILImage mode=RGB size=200x200, TensorCategory(0)),(PILImage mode=RGB size=200x200, TensorCategory(2)),(PILImage mode=RGB size=200x200, TensorCategory(0)),(PILImage mode=RGB size=200x200, TensorCategory(2)),(PILImage mode=RGB size=200x200, TensorCategory(0)),(PILImage mode=RGB size=200x200, TensorCategory(2)),(PILImage mode=RGB size=200x200, TensorCategory(0)),(PILImage mode=RGB size=200x200, TensorCategory(0))...]

In [28]:
test_im = get_image_files(config["ds_path"])

In [29]:
captum.visualize(test_im[0])

RuntimeError: grid_sampler(): expected grid to have size 1 in last dimension, but got grid with sizes [3, 224, 224, 2]